# Segmentation and Clustering Neighborhoods in Toronto

## Part I - building dataframe from website

specify url that contains the dataset

In [2]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url,header=0)
len(df)
df_trt = df[0]
df_trt.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
# pip install lxml

read tables from webpage. There are 3 tables in the webpage. Assign the appropriate table to df_trt

Drop cells where column Borough is 'Not assigned' and reset dataframe index

In [3]:
NotAss=df_trt[df_trt['Borough']=='Not assigned']
df_trt.drop(NotAss.index, inplace = True)
df_trt.reset_index(inplace=True)
df_trt.drop(['index'],axis=1,inplace=True)
df_trt.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df_trt.shape

(103, 3)

In [4]:
!pip install geocoder


## Part II - getting lattitude and longitude coordinates

Coded provided from instruction to get coordinates using Geocoder Python Package

In [119]:
import geocoder
from geopy.geocoders import Nominatim

/*for code in df_trt['Postal Code']:
    print(code)
    #lat_lng_coords = None
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print(latitude)
    print(longitude)
*/

Geocoder Python Package not reliable, coordinates from csv file used instead

In [5]:
!wget -q -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

df_coord = pd.read_csv('Geospatial_data.csv')
df_coord.head()

Data downloaded!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
#set the Postal Code columns as the index name so that it's easier to locate to one specific postal code cell
df_trt.set_index('Postal Code', inplace = True)
df_coord.set_index('Postal Code', inplace = True)

#adding coordinates info to df_trt dataframe

for code in df_trt.index:
    df_trt.loc[code,'Latitude'] = df_coord.loc[code,'Latitude']
    df_trt.loc[code,'Longitude'] = df_coord.loc[code,'Longitude']
    
#reset index
df_trt.reset_index(inplace=True)
df_trt.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Part III - explore and cluster neighborhood

Get Foursquare credencials and url

In [8]:
import json
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

CLIENT_ID = 'PKUTNEG4HV4S4KXTGXOJMW0OFPXR1ZNL3JZJUAHPIFM1E3BS'
CLIENT_SECRET = '1ANQ5XBNYQ3JA4OKCG2NRVNN0JBH1PNCV44Q5PAKH5TIJLR2'
VERSION = '20180605'

In [170]:
df_trt[df_trt['Neighborhood'] == 'Toronto Dominion Centre, Design Exchange']

,Postal Code,Borough,Neighborhood,Latitude,Longitude
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


In [175]:
radius = 900
LIMIT=100

neighborhood_latitude = df_trt.loc[0,'Latitude']
neighborhood_longitude= df_trt.loc[0,'Longitude']

neighborhood_latitude = 43.647177
neighborhood_longitude = -79.381576
url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,
                                                                                                                     CLIENT_SECRET,
                                                                                                                     VERSION,
                                                                                                                     neighborhood_latitude,
                                                                                                                    neighborhood_longitude,
                                                                                                                        radius,
                                                                                                                            LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=PKUTNEG4HV4S4KXTGXOJMW0OFPXR1ZNL3JZJUAHPIFM1E3BS&client_secret=1ANQ5XBNYQ3JA4OKCG2NRVNN0JBH1PNCV44Q5PAKH5TIJLR2&v=20180605&ll=43.647177,-79.381576&radius=900&limit=100'

In [ ]:
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id=PKUTNEG4HV4S4KXTGXOJMW0OFPXR1ZNL3JZJUAHPIFM1E3BS&client_secret=1ANQ5XBNYQ3JA4OKCG2NRVNN0JBH1PNCV44Q5PAKH5TIJLR2&v=20180605&ll=43.789053,-79.40849279999999&radius=600&limit=100'
url='https://api.foursquare.com/v2/venues/explore?&client_id=PKUTNEG4HV4S4KXTGXOJMW0OFPXR1ZNL3JZJUAHPIFM1E3BS&client_secret=1ANQ5XBNYQ3JA4OKCG2NRVNN0JBH1PNCV44Q5PAKH5TIJLR2&v=20180605&ll=43.7532586,-79.3296565&radius=800&limit=100'
results = requests.get(url).json()
print(results)
nearby_venues = json_normalize(results['response']['groups'][0]['items'])

In [ ]:
nearby_venues

In [194]:
def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

        # make the GET request
        print(url)
        results = requests.get(url).json()["response"]['groups'][0]['items']    
        #if name == 'Willowdale, Newtonbrook':
            
         #   print(results)
           # print('------------')
      
        # return only relevant information for each nearby venue
        venues_list.append([( name, lat, lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 
                  'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)
    
    

In [ ]:
LIMIT=100
Toronto_nearby_venues = getNearbyVenues(df_trt['Neighborhood'],df_trt['Latitude'],df_trt['Longitude'])

In [160]:
Toronto_nearby_venues.shape
#Toronto_nearby_venues.head()

(3981, 7)

In [162]:
print('There are {} uniques categories.'.format(len(Toronto_nearby_venues['Venue Category'].unique())))
count = Toronto_nearby_venues.groupby('Neighborhood').count()

There are 340 uniques categories.


In [163]:
df_trt.sort_values('Neighborhood').head(5)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
78,M1S,Scarborough,Agincourt,43.794200,-79.262029
93,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
39,M2K,North York,Bayview Village,43.786947,-79.385975
55,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750


In [164]:
# one hot encoding
trt_onehot = pd.get_dummies(Toronto_nearby_venues[['Venue Category']], prefix="", prefix_sep="")

trt_onehot.drop('Neighborhood',axis=1, inplace=True)
trt_onehot['Neighborhood'] = Toronto_nearby_venues['Neighborhood'] 

fixed_columns = [trt_onehot.columns[-1]] + list(trt_onehot.columns[:-1])
trt_onehot = trt_onehot[fixed_columns]

trt_onehot.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
#group by neighborhood, take mean of each category
trt_grouped = trt_onehot.groupby('Neighborhood').mean().reset_index()
trt_grouped.head()


,Neighborhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.025641,0.0,0.0,0.0,0.0,0.0,0.025641,0.0,0.0


In [166]:
trt_grouped.shape

(98, 340)

In [146]:
temp = trt_grouped.loc[0]
temp.iloc[1:].sort_values(ascending = False).head(5)

Lounge                       0.111111
Coffee Shop                  0.111111
Badminton Court              0.111111
Latin American Restaurant    0.111111
Skating Rink                 0.111111
Name: 0, dtype: object

In [43]:
# find top 5 veunes for each neighborhood
venue_limit = 5
for row in trt_grouped.iterrows():
    print('--------'+row[1][0]+'-----------')
    print(row[1][1:].sort_values(ascending = False).head(venue_limit))
    print('\n')

--------Agincourt-----------
Lounge                       0.166667
Clothing Store               0.166667
Latin American Restaurant    0.166667
Breakfast Spot               0.166667
Skating Rink                 0.166667
Name: 0, dtype: object


--------Alderwood, Long Branch-----------
Pizza Place          0.2
Coffee Shop          0.1
Pool                 0.1
Convenience Store    0.1
Gas Station          0.1
Name: 1, dtype: object


--------Bathurst Manor, Wilson Heights, Downsview North-----------
Bank                 0.0909091
Coffee Shop          0.0909091
Pizza Place          0.0454545
Mobile Phone Shop    0.0454545
Restaurant           0.0454545
Name: 2, dtype: object


--------Bayview Village-----------
Chinese Restaurant     0.25
Bank                   0.25
Café                   0.25
Japanese Restaurant    0.25
Event Space               0
Name: 3, dtype: object


--------Bedford Park, Lawrence Manor East-----------
Coffee Shop            0.111111
Italian Restaurant     0.111111


In [214]:
columns = ['Neighborhood']
date = ['st','nd','rd']
for i in range (1,11):
    try:
        columns.append('{}{} venue'.format(i,date[i-1]))
    except:
        columns.append('{}th venue'.format(i))

trt_neighbor = pd.DataFrame(columns=columns)
trt_neighbor['Neighborhood'] = trt_grouped['Neighborhood']


In [215]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


for ind in range(trt_grouped.shape[0]):
    trt_neighbor.iloc[ind, 1:] = return_most_common_venues(trt_grouped.iloc[ind, :], 10)


In [216]:
from sklearn.cluster import KMeans
import numpy as np # library to handle data in a vectorized manner

# number of clusters
k = 10

trt_grouped_clustering = trt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(trt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 7, 7, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [203]:
df_trt.shape

(103, 5)

In [217]:
# add clustering labels
trt_neighbor.insert(0, 'Cluster Labels', kmeans.labels_)
trt_merged = df_trt

# merge to add latitude/longitude for each neighborhood
trt_merged = trt_merged.join(trt_neighbor.set_index('Neighborhood'), on='Neighborhood')

trt_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st venue,2nd venue,3rd venue,4th venue,5th venue,6th venue,7th venue,8th venue,9th venue,10th venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,5.0,Bus Stop,Park,Road,Food & Drink Shop,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dive Bar
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Park,Portuguese Restaurant,Pizza Place,Coffee Shop,Hockey Arena,Intersection,Café,Yoga Studio,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Restaurant,Park,Italian Restaurant,Theater,Pub,Bakery,Café,Breakfast Spot,Gym / Fitness Center
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Vietnamese Restaurant,Fried Chicken Joint,Dessert Shop,Coffee Shop,Fast Food Restaurant,Athletics & Sports,Accessories Store,Seafood Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Bubble Tea Shop,Italian Restaurant,Burger Joint,Gastropub,Sushi Restaurant,Café,French Restaurant,Bookstore,Park


In [210]:
trt_merged.shape

(103, 16)

In [218]:
#print(trt_merged[trt_merged['1st venue']=='NaN'])
check=trt_merged.isna().any(axis=1)
trt_merged[check]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st venue,2nd venue,3rd venue,4th venue,5th venue,6th venue,7th venue,8th venue,9th venue,10th venue
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
!pip install folium

In [219]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[43.65, -79.38], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



In [220]:
# add markers to the map

markers_colors = []
for lat, lon, poi, cluster in zip(trt_merged['Latitude'], trt_merged['Longitude'], trt_merged['Neighborhood'], trt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if np.isnan(cluster):
        cluster = 4
        print(poi)
    #print(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Upper Rouge
